In [1]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)


In [6]:
import os
import cv2
import mediapipe as mp
classified_pose = kNN("tadasana_prueba.png",5)
print(classified_pose)

tadasana


In [2]:
from collections import Counter
def kNN(pose_path, k, test_csvs = []):
    image = cv2.imread(pose_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if not results.pose_landmarks:
        return "empty"

    landmarks = {lm_idx: (lm.x, lm.y, lm.z) for lm_idx, lm in enumerate(results.pose_landmarks.landmark)}

    image_vectors = {}

    landmark_names = mp_pose.PoseLandmark

    for connection in mp_pose.POSE_CONNECTIONS:
        start_idx, end_idx = connection

        start_name = landmark_names(start_idx).name
        end_name = landmark_names(end_idx).name

        vector_name = f"{start_name}_{end_name}"

        x1, y1, z1 = landmarks[start_idx]
        x2, y2, z2 = landmarks[end_idx]

        vec_x = x2 - x1
        vec_y = y2 - y1
        vec_z = z2 - z1

        image_vectors[vector_name] = (vec_x, vec_y, vec_z)

    distances = []
    for pose_folder in os.listdir("dataset_vectors"):
        folder_path = os.path.join("dataset_vectors", pose_folder)
        for pose_csv in os.listdir(folder_path):
            csv_path = os.path.join(folder_path, pose_csv)
            
            if csv_path in test_csvs:
                continue
                
            csv = pd.read_csv(csv_path)
            distance = 0
            
            for vector_name, image_vector in image_vectors.items():
                subset = csv[csv["name"] == vector_name]
                if subset.empty:
                    continue
                for _, row in subset.iterrows():
                    csv_vector = (row["x"], row["y"], row["z"])
                    distance += cosine_distance(csv_vector, image_vector)

            distances.append((distance, pose_folder))

    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]

    classification_scores = Counter([folder for _, folder in k_nearest])
    max_count = max(classification_scores.values())
    
    most_common_classes = [cls for cls, count in classification_scores.items() if count == max_count]

    predicted_class = most_common_classes if len(most_common_classes) > 1 else most_common_classes[0]
    return predicted_class
    
def cosine_distance(vec1, vec2):
    v1 = np.array(vec1)
    v2 = np.array(vec2)
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-9)  
    return 1 - abs(cos_sim)

In [5]:
import random
test_percentage = 0.1
test_paths = []

for pose_folder in os.listdir("dataset"):
    images = []
    folder_path = os.path.join("dataset", pose_folder)
    for pose_csv in os.listdir(folder_path):
        csv_path = os.path.join(folder_path, pose_csv)
        images.append(csv_path)
        
    test_size = int(len(images) * test_percentage)
    test_sample = random.sample(images, test_size)
    test_paths.extend(test_sample)
    
print(test_paths)

['dataset\\adho mukha svanasana\\67. downward-facing-dog-adho-mukha-svanasana.png', 'dataset\\adho mukha svanasana\\1. 5-benefits-of-downward-facing-dog-pose.png', 'dataset\\adho mukha svanasana\\4. downward-facing-dog.png', 'dataset\\adho mukha svanasana\\33. screenshot_2015-02-10-12-04-46.png', 'dataset\\adho mukha svanasana\\10. screen-shot-2017-09-15-at-17.00.06-1024x585.png', 'dataset\\adho mukha svanasana\\92. down-dog.png', 'dataset\\adho mukha vriksasana\\40-0.png', 'dataset\\adho mukha vriksasana\\25-0.png', 'dataset\\adho mukha vriksasana\\17-0.png', 'dataset\\adho mukha vriksasana\\77-0.png', 'dataset\\adho mukha vriksasana\\46-1.png', 'dataset\\agnistambhasana\\38-0.png', 'dataset\\agnistambhasana\\12-0.png', 'dataset\\agnistambhasana\\26-0.png', 'dataset\\ananda balasana\\90-0.png', 'dataset\\ananda balasana\\25-0.png', 'dataset\\ananda balasana\\12-0.png', 'dataset\\ananda balasana\\87-0.png', 'dataset\\ananda balasana\\42-0.png', 'dataset\\anantasana\\28-1.png', 'dataset

In [6]:
# Inicializar k (cantidad de vecinos)
k = 5
true_predicted=[]
false_predicted=[]
test_csvs = [image.replace(".png", ".csv").replace(".jpg", ".csv") for image in test_paths]
for image_path in test_paths:
    predicted_class=kNN(image_path,k,test_csvs)
    real_class = os.path.basename(os.path.dirname(image_path))
    if predicted_class == real_class:
        true_predicted.append(image_path)
    else:
        false_predicted.append(image_path)

print('True predicted:')
print(len(true_predicted))
print('False predicted:')
print(len(false_predicted))
#5%-> 162t 82f
#10% - 381t 169f


True predicted:
381
False predicted:
169
